# Colab Start Section

Step 1 is to upload a copy of your private key. I am assuming your private key is named id_ed25519. If that isn't the case, you'll have to make manual changes.

In [43]:
# Change these of course to what is your email and your GitHub username
EMAIL = "pauleanderson@gmail.com"
USERNAME = "pauleanderson"

In [44]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [45]:
%%bash -s "$EMAIL" "$USERNAME"

rm -rf /root/.ssh/*
mkdir /root/.ssh

mv id_ed25519 /root/.ssh/

chmod 600 /root/.ssh/id_ed25519

git config --global user.email "$1"
git config --global user.name "$2"

mkdir: /root: No such file or directory
mv: rename id_ed25519 to /root/.ssh/: No such file or directory
chmod: /root/.ssh/id_ed25519: No such file or directory


In [46]:
%%bash -s "$USERNAME"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

git clone git@github.com:anderson-fall-2024/csc-487-$1.git

Agent pid 18995


Identity added: /Users/pander14/.ssh/id_ed25519 (pauleanderson@gmail.com)
# github.com:22 SSH-2.0-babeld-cda406d09
Cloning into 'csc-487-pauleanderson'...


In [66]:
%load_ext autoreload
%autoreload 2


from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

import sys
sys.path.insert(0,f"csc-487-{USERNAME}")
sys.path.insert(0,"..")

import py487

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [68]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on a hardware accelerator like the
GPU or MPS, if available. Let\'s check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are
available, otherwise we use the CPU.


In [69]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


Define the Class
================

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [70]:
torch.manual_seed(1)
model = py487.tutorials.NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [71]:
torch.manual_seed(1)
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")
print(f"Probabilities: {pred_probab}")

Predicted class: tensor([7], device='mps:0')
Probabilities: tensor([[0.1004, 0.1044, 0.0890, 0.0991, 0.0998, 0.0955, 0.0926, 0.1111, 0.1069,
         0.1011]], device='mps:0', grad_fn=<SoftmaxBackward0>)


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [72]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [73]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [74]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [75]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3622, -0.3701, -0.1414,  0.6163,  0.3905,  0.2006, -0.2799, -0.5290,
         -0.0490,  0.4167,  0.1413, -0.2661,  0.5620,  0.7572, -0.2707,  0.0926,
          0.4475,  0.1415,  0.4279,  0.1412],
        [-0.4611, -0.5728, -0.3562,  0.8147, -0.0936,  0.1774, -0.3873, -0.5876,
          0.0043,  0.3577, -0.1779, -0.5675,  0.3810,  0.4533,  0.1013, -0.1296,
          0.4849,  0.3524,  0.3930,  0.1900],
        [-0.5086, -0.4941,  0.1930,  0.3946,  0.2976,  0.4015, -0.5687, -0.9443,
          0.1023,  0.3193, -0.0084, -0.0032,  0.2614,  0.5684, -0.0843, -0.0589,
          0.3377, -0.2759,  0.5410,  0.2048]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.6163, 0.3905, 0.2006, 0.0000, 0.0000, 0.0000,
         0.4167, 0.1413, 0.0000, 0.5620, 0.7572, 0.0000, 0.0926, 0.4475, 0.1415,
         0.4279, 0.1412],
        [0.0000, 0.0000, 0.0000, 0.8147, 0.0000, 0.1774, 0.0000, 0.0000, 0.0043,
         0.3577, 0.0000, 0.0000, 0.3810, 0.4533, 0.10

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [76]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [77]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [78]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0184, -0.0158, -0.0069,  ...,  0.0068, -0.0041,  0.0025],
        [-0.0274, -0.0224, -0.0309,  ..., -0.0029,  0.0013, -0.0167]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0126, -0.0036], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0176,  0.0350,  0.0320,  ...,  0.0425,  0.0170,  0.0101],
        [ 0.0020, -0.0058, -0.0099,  ..., -0.0371,  0.0146,  0.0011]],
       device='mps:0', grad_fn=<Slice

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)


In [79]:
# Good job!
# Woohoo!

# Test your code

In [81]:
%%bash -s "$USERNAME"

cd csc-487-$1/tests
pytest test_buildmodel_tutorial.py

============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-8.3.3, pluggy-1.5.0
rootdir: /Users/pander14/GitHub/csc-487-instructor/tests
plugins: anyio-4.6.0
collected 1 item

test_buildmodel_tutorial.py F                                            [100%]

=================================== FAILURES ===================================
_______________________________ test_exercise_1 ________________________________

    def test_exercise_1():
        torch.manual_seed(1)
        model = py487.tutorials.NeuralNetwork().to(device)
        torch.manual_seed(1)
        X = torch.rand(1, 28, 28, device=device)
        logits = model(X)
        pred_probab = nn.Softmax(dim=1)(logits)
        your_answer = pred_probab.tolist()
>       assert np.all(np.around(answers['exercise_1'],4) == np.around(your_answer,4))
E       NameError: name 'np' is not defined

test_buildmodel_tutorial.py:41: NameError
=========================== short te

CalledProcessError: Command 'b'\ncd ../tests #csc-487-$1/tests\npytest test_buildmodel_tutorial.py\n'' returned non-zero exit status 1.

# Colab End Section - Submit your code

In [ ]:
%%bash -s "$USERNAME"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

cd csc-448-$1
./command_line_sync.sh